## Chapter12.Traits




### How traits work




In [2]:
trait Philosophical {
    def philosophize() = {
      println("I consume memory, therefore I am!")
    }
}

In [3]:
class Frog extends Philosophical {
    override def toString = "green"
}
val frog = new Frog
frog.philosophize()
val phil: Philosophical = frog
phil.philosophize()

I consume memory, therefore I am!
I consume memory, therefore I am!


In [15]:
class Animal

class Frog extends Animal with Philosophical {
    override def toString = "green"
}
val frog = new Frog

In [16]:
class Animal
  trait HasLegs

  class Frog extends Animal with Philosophical with HasLegs {
    override def toString = "green"
}

In [17]:
class Animal

class Frog extends Animal with Philosophical {
    override def toString = "green"
    override def philosophize() = {
      println("It ain't easy being " + toString + "!")
    }
}
val phrog: Philosophical = new Frog
phrog.philosophize()

It ain't easy being green!


In [18]:
object Ex2 {
  class Animal
  abstract class IntQueue {
    def get: Int
    def put(x: Int)
  class Frog extends Animal with Philosophical {
    override def toString = "green"
    override def philosophize() = {
      println("It ain't easy being " + toString + "!")
    }
  }

  class Point(x: Int, y: Int)
  trait CharSequence {
    def charAt(index: Int): Char
    def length: Int
    def subSequence(start: Int, end: Int): CharSequence
    def toString(): String
  }
  trait Ordered[T] {
    def compare(that: T): Int
  
    def <(that: T): Boolean = (this compare that) < 0
    def >(that: T): Boolean = (this compare that) > 0
    def <=(that: T): Boolean = (this compare that) <= 0
    def >=(that: T): Boolean = (this compare that) >= 0
  }

  import scala.collection.mutable.ArrayBuffer
  
  class BasicIntQueue extends IntQueue {
    private val buf = new ArrayBuffer[Int]
    def get() = buf.remove(0)
    def put(x: Int) = { buf += x }
  }
  trait Incrementing extends IntQueue {
    abstract override def put(x: Int) = { super.put(x + 1) }
  }
  trait Doubling extends IntQueue {
    abstract override def put(x: Int) = { super.put(2 * x) }
  }

  def main(args: Array[String]): Unit = {
    (new Frog).philosophize()

    // Multiple inheritance thought experiment
    val q = new BasicIntQueue with Incrementing with Doubling
    q.put(42)  // which put would be called?
    println("q [" + q + "]")
  }
}
}


### Thin versus rich interfaces




In [5]:
trait CharSequence {
    def charAt(index: Int): Char
    def length: Int
    def subSequence(start: Int, end: Int): CharSequence
    def toString(): String
}

Error: invalid syntax (18)

### Example: Rectangular objects



In [6]:
class Point(val x: Int, val y: Int)

class Rectangle(val topLeft: Point, val bottomRight: Point) {
    def left = topLeft.x
    def right = bottomRight.x
    def width = right - left
    // and many more geometric methods...
}
abstract class Component {
    def topLeft: Point
    def bottomRight: Point

    def left = topLeft.x
    def right = bottomRight.x
    def width = right - left
    // and many more geometric methods...
}

Error: invalid syntax (17)

In [20]:
trait Rectangular {
    def topLeft: Point
    def bottomRight: Point

    def left = topLeft.x
    def right = bottomRight.x
    def width = right - left
    // and many more geometric methods...
  }


abstract class Component extends Rectangular {
    // other methods...
  }


// In file traits/Rectangle.scala

class Rectangle(val topLeft: Point, val bottomRight: Point)
      extends Rectangular {

    // other methods...
}
rect.left
rect.right
rect.width

Error: invalid syntax (17)

### The Ordered trait




In [8]:
class Rational(n: Int, d: Int) {
    // ...
    def < (that: Rational) = 
      this.numer * that.denom < that.numer * this.denom
    def > (that: Rational) = that < this
    def <= (that: Rational) = (this < that) || (this == that)
    def >= (that: Rational) = (this > that) || (this == that)
  }


In [9]:
trait Ordered[T] {
    def compare(that: T): Int
  
    def <(that: T): Boolean = (this compare that) < 0
    def >(that: T): Boolean = (this compare that) > 0
    def <=(that: T): Boolean = (this compare that) <= 0
    def >=(that: T): Boolean = (this compare that) >= 0
}


class Rational(n: Int, d: Int) extends Ordered[Rational] {
  def compare(that: Rational) =
    (this.numer * that.denom) - (that.numer * this.denom)

  require(d != 0)

  private val g = gcd(n.abs, d.abs)
  val numer = n / g
  val denom = d / g

  def this(n: Int) = this(n, 1)

  def + (that: Rational): Rational =
    new Rational(
      numer * that.denom + that.numer * denom,
      denom * that.denom
    )

  def + (i: Int): Rational =
    new Rational(numer + i * denom, denom)

  def - (that: Rational): Rational =
    new Rational(
      numer * that.denom - that.numer * denom,
      denom * that.denom
    )

  def - (i: Int): Rational =
    new Rational(numer - i * denom, denom)

  def * (that: Rational): Rational =
    new Rational(numer * that.numer, denom * that.denom)

  def * (i: Int): Rational =
    new Rational(numer * i, denom)

  def / (that: Rational): Rational =
    new Rational(numer * that.denom, denom * that.numer)

  def / (i: Int): Rational =
    new Rational(numer, denom * i)

  override def toString = s"$numer/$denom"

  private def gcd(a: Int, b: Int): Int = 
    if (b == 0) a else gcd(b, a % b)

  override def equals(other: Any): Boolean =
    other match {

      case that: Rational =>
        (that canEqual this) &&
        numer == that.numer &&
        denom == that.denom

      case _ => false
    }

  def canEqual(other: Any): Boolean =
    other.isInstanceOf[Rational]

  override def hashCode: Int = (numer, denom).##

}

val half = new Rational(1, 2)
val third = new Rational(1, 3)
half < third
half > third

true

### Traits as stackable modifications




In [11]:
abstract class IntQueue {
    def get(): Int
    def put(x: Int): Unit
  }
import scala.collection.mutable.ArrayBuffer

class BasicIntQueue extends IntQueue {
    private val buf = new ArrayBuffer[Int]
    def get() = buf.remove(0)
    def put(x: Int) = { buf += x }
}
val queue = new BasicIntQueue
queue.put(10)
queue.put(20)
queue.get()
queue.get()

20

In [12]:
trait Doubling extends IntQueue {
    abstract override def put(x: Int) = { super.put(2 * x) }
}
class MyQueue extends BasicIntQueue with Doubling
val queue = new MyQueue
queue.put(10)
queue.get()
val queue2 = new BasicIntQueue with Doubling
queue2.put(10)
queue2.get()

20

In [13]:
trait Incrementing extends IntQueue {
    abstract override def put(x: Int) = { super.put(x + 1) }
  }
  trait Filtering extends IntQueue {
    abstract override def put(x: Int) = {
      if (x >= 0) super.put(x)
    }
}
val queue = (new BasicIntQueue with Incrementing with Filtering)
queue.put(-1); queue.put(0); queue.put(1)
queue.get()
queue.get()

2

### Why not multiple inheritance?



In [21]:
// Multiple inheritance thought experiment
val q = new BasicIntQueue with Incrementing with Doubling
q.put(42)  // which put would be called?


// Multiple inheritance thought experiment
trait MyQueue extends BasicIntQueue
    with Incrementing with Doubling {

def put(x: Int) = {
    Incrementing.super.put(x) // (Not real Scala)
    Doubling.super.put(x)
}
}


class Animal 
trait Furry extends Animal
trait HasLegs extends Animal
trait FourLegged extends HasLegs
class Cat extends Animal with Furry with FourLegged

Error: Incrementing is not an enclosing class (296)Error: value put is not a member of {} (315)Error: Doubling is not an enclosing class (347)Error: value put is not a member of {} (362)